In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Not sure where the unnamed column came from but just drop it
df = pd.read_csv('../data/processed_data.csv').drop(u'Unnamed: 0', axis=1) 
df = df.dropna()
print(df.shape)
print(df.columns)
df.head()

(25697, 13)
Index([u'season', u'Opponent DRtg/A', u'shot_distance', u'left_side',
       u'middle', u'right_side', u'moving', u'playoffs', u'home', u'segababa',
       u'Dunk or Layup', u'shot_made_flag', u'total_seconds_remaining'],
      dtype='object')


,season,Opponent DRtg/A,shot_distance,left_side,middle,right_side,moving,playoffs,home,segababa,Dunk or Layup,shot_made_flag,total_seconds_remaining
0,1,108.06,256.0,0,0,1,0,0,1,0,0,0.0,2202
1,1,101.68,228.0,0,0,1,0,0,0,0,0,0.0,2048
2,1,109.65,323.0,0,0,1,0,0,0,1,0,1.0,1957
3,1,109.65,0.0,0,1,0,0,0,0,1,0,0.0,1834
4,1,109.65,148.0,0,1,0,0,0,0,1,0,1.0,1767


In [2]:
df_submit = pd.read_csv('../data/submit_data.csv').drop(u'Unnamed: 0', axis=1)
print df_submit.shape

X_submit = df_submit.drop(['shot_id', 'shot_made_flag'], axis=1)

(5000, 14)


,shot_id,season,Opponent DRtg/A,shot_distance,left_side,middle,right_side,moving,playoffs,home,segababa,Dunk or Layup,shot_made_flag,total_seconds_remaining
0,22907,1,108.57,102.0,0,1,0,0,0,0,0,0,NaN,1878
1,22909,1,108.57,0.0,0,1,0,0,0,0,0,1,NaN,925
2,22927,1,109.44,256.0,0,0,1,0,0,0,0,0,NaN,1811
3,22926,1,109.44,15.0,1,0,0,0,0,0,0,0,NaN,1860
4,22930,1,109.44,250.0,0,1,0,0,0,0,0,0,NaN,1538


In [3]:
y = df['shot_made_flag']
X = df.drop('shot_made_flag', axis=1)

## Some more initializations...

In [22]:
from sklearn.model_selection import KFold
seed = 7
processors=1
num_folds = 3
scoring='neg_log_loss'
kfold = KFold(n_splits=num_folds, random_state=seed)

## Use this function to create a submission

In [ ]:
def create_submission(model_grid):
    predictions = model_grid.predict_proba(X_submit)
    submission = pd.DataFrame()
    submission["shot_id"] = df_submit['shot_id']
    submission["shot_made_flag"] = predictions.T[1]
    submission = submission.sort_values(['shot_id'], ascending=True)
    return submission

## XGBoost

In [4]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}

xg_grid = GridSearchCV(
    xgb.XGBClassifier(**ind_params),
    cv_params,
    scoring = scoring, 
    cv=3, n_jobs=-1
)


xg_grid.fit(X, y)

Done


### Create a submission

In [14]:
submission = create_submission(xg_grid)
submission.head()

,shot_id,shot_made_flag
694,1,0.361415
695,8,0.337337
697,17,0.741243
696,20,0.739300
705,33,0.358755


In [16]:
submission.to_csv("../data/submission_XGBoost_proba.csv",index=False)

## Linear Discriminant Analysis

In [24]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda_grid = GridSearchCV(
    estimator = LinearDiscriminantAnalysis(),
    param_grid = {
        'solver': ['lsqr'],
        'shrinkage': [0, 0.25, 0.5, 0.75, 1],
        'n_components': [None, 2, 5, 10]
    },
    cv = kfold,
    scoring = scoring,
    n_jobs = processors)

lda_grid.fit(X, y)

GridSearchCV(cv=KFold(n_splits=3, random_state=7, shuffle=False),
       error_score='raise',
       estimator=LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'shrinkage': [0, 0.25, 0.5, 0.75, 1], 'n_components': [None, 2, 5, 10], 'solver': ['lsqr']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

### Create a submission

In [27]:
submissions = create_submission(lda_grid)
submission.head()

,shot_id,shot_made_flag
694,1,0.374771
695,8,0.396593
697,17,0.795422
696,20,0.784548
705,33,0.392260


In [28]:
submission.to_csv("../data/submission_lda.csv",index=False)

## Logistic Regression

In [47]:
from sklearn.linear_model import LogisticRegression

lr_grid = GridSearchCV(
    estimator = LogisticRegression(random_state=seed),
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 1, 10, 100, 1000],        
    },
    cv = kfold,
    scoring = scoring,
    n_jobs = processors)

lr_grid.fit(X, y)

GridSearchCV(cv=KFold(n_splits=3, random_state=7, shuffle=False),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=7, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

### Create a submission

In [33]:
submission = create_submission(lr_grid)
submission.head()

,shot_id,shot_made_flag
694,1,0.382450
695,8,0.401110
697,17,0.802239
696,20,0.791596
705,33,0.399008


In [34]:
submission.to_csv("../data/submission_log_r.csv",index=False)

## Random Forest

In [39]:
from sklearn.ensemble import RandomForestClassifier
rf_grid = GridSearchCV(
    estimator = RandomForestClassifier(warm_start=True, random_state=seed),
    param_grid = {
        'n_estimators': [100, 200],
        'criterion': ['gini', 'entropy'],
        'max_depth': [8, 10],
        'bootstrap': [True]
    }, 
    cv = kfold, 
    scoring = scoring, 
    n_jobs = processors)

rf_grid.fit(X, y)

GridSearchCV(cv=KFold(n_splits=3, random_state=7, shuffle=False),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=7,
            verbose=0, warm_start=True),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200], 'bootstrap': [True], 'criterion': ['gini', 'entropy'], 'max_depth': [8, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [41]:
submission = create_submission(rf_grid)
submission.head()

,shot_id,shot_made_flag
694,1,0.384379
695,8,0.380619
697,17,0.718616
696,20,0.713774
705,33,0.391916


In [42]:
submission.to_csv("../data/submission_rf.csv",index=False)

## Gradient Boosting

In [44]:
from sklearn.ensemble import GradientBoostingClassifier

gbm_grid = GridSearchCV(
    estimator = GradientBoostingClassifier(warm_start=True, random_state=seed),
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [2, 3, 4],
        'learning_rate': [1e-1, 1]
    }, 
    cv = kfold, 
    scoring = scoring, 
    n_jobs = processors)

gbm_grid.fit(X, y)

GridSearchCV(cv=KFold(n_splits=3, random_state=7, shuffle=False),
       error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=7,
              subsample=1.0, verbose=0, warm_start=True),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200], 'learning_rate': [0.1, 1], 'max_depth': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [45]:
submission = create_submission(gbm_grid)
submission.head()

,shot_id,shot_made_flag
694,1,0.401024
695,8,0.385624
697,17,0.788284
696,20,0.771940
705,33,0.395484


In [46]:
submission.to_csv("../data/submission_gbc.csv",index=False)